In [1]:
from activation_additions import prompt_utils, hook_utils, adhoc_actadds, adhoc_hyperparameter_search
import numpy as np
from contextlib import contextmanager
from typing import Tuple, Callable, Optional

import numpy as np
import torch as t
from torch import nn
from transformers import GenerationConfig, LlamaForCausalLM, LlamaTokenizer

import accelerate
import importlib

MODEL_DIR: str = "/mnt/ssd-2/mesaoptimizer/llama/hf/7B"
MAX_NEW_TOKENS: int = 50
NUM_CONTINUATIONS: int = 5
SEED: int = 0
DO_SAMPLE: bool = True
TEMPERATURE: float = 1.0
TOP_P: float = 0.9
REP_PENALTY: float = 2.0
PLUS_PROMPT, MINUS_PROMPT = "Harry Potter ", " Lord of the Rings"
CHAT_PROMPT: str = "My favourite book is "
ACT_NUM: int = 6
COEFF: int = 4

sampling_kwargs: dict = {
    "temperature": TEMPERATURE,
    "top_p": TOP_P,
    "repetition_penalty": REP_PENALTY,
}

# Set torch and numpy seeds.
t.manual_seed(SEED)
np.random.seed(SEED)


In [25]:
t.set_grad_enabled(False)
# An accelerate wrapper does all the parallelization across devices.
accelerator = accelerate.Accelerator()
model = LlamaForCausalLM.from_pretrained(MODEL_DIR, device_map="cuda:0")
tokenizer = LlamaTokenizer.from_pretrained(MODEL_DIR)
model, tokenizer = accelerator.prepare(model, tokenizer)
model.tie_weights()
# model.half()
model.eval()

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

OutOfMemoryError: CUDA out of memory. Tried to allocate 172.00 MiB (GPU 0; 47.35 GiB total capacity; 35.70 GiB already allocated; 129.19 MiB free; 35.71 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [10]:
#Define the kind of behaviour you want: for example: speaking british instead of american english
#a set of [prompt, wanted completion, unwanted completion]

US_UK_dict = {
    "1": ["The delivery driver parked the", " lorry outside the warehouse.", " truck outside the warehouse."],
    "2": ["Can you please turn off the", " tap in the kitchen?", " faucet in the kitchen?"],
    "3": ["I recently rented my first", " flat.", " apartment."],
    "4": ["We are planning to go on a", " holiday to Europe next summer.", " vacation to Europe next summer."],
    "5": ["Would you like a", " biscuit with your tea?", " cookie with your tea?"],
    "6": ["I'm thinking of making a dish with", " aubergine tonight.", " eggplant tonight."],
    "7": ["It's getting chilly; I'll wear my", " jumper today.", " sweater today."],
    "8": ["She bought a bag of salt and vinegar", " crisps from the store.", " chips from the store."],
    "9": ["It's too dark here. Do you have a", " torch?", " flashlight?"],
    "10": ["My kids enjoy playing", " football every weekend.", " soccer every weekend."]
}
prompts=[]
UK_completions=[]
US_completions=[]
for key, example in US_UK_dict.items():
    prompts.append(example[0])
    UK_completions.append(example[1])
    US_completions.append(example[2])

In [14]:
from importlib import reload

# Reload the modules
adhoc_actadds = reload(adhoc_actadds)
adhoc_hyperparameter_search = reload(adhoc_hyperparameter_search)


In [17]:
resolution=10
#when you have your steerig vector(here for example: "British" - "American"), do a grid search over layers and coefficents
#this might take a while
weighted_steering_prompts={" British":1," American":-1}
layer_list=[int(i) for i in np.linspace(1,len(model.model.layers)-1,resolution)]
coefficent_list= [i for i in np.linspace(0,10,resolution,dtype=int)]

df=adhoc_hyperparameter_search.layer_coefficient_gridsearch(
    model,
    tokenizer,
    prompts,
    weighted_steering_prompts,
    layer_list,
    coefficent_list,
    UK_completions,
    US_completions
)

Processing: 100%|██████████| 100/100 [06:30<00:00,  3.90s/it]


In [18]:
wanted_matrix, unwanted_matrix=adhoc_hyperparameter_search.create_perplexity_matrices(df)
adhoc_hyperparameter_search.plot_matrices_with_pareto(df)
adhoc_hyperparameter_search.plot_pareto_frontier(df)

In [19]:
Loving_Hating_dict = {
    "1": ["The painting in the living room", " brings warmth and joy to our home.", " is a damn eyesore and should be trashed."],
    "2": ["The song you sang at the party", " had everyone mesmerized.", " was a pathetic screech-fest."],
    "3": ["That cafe you introduced me to", " has the best desserts I've ever tasted.", " was a total rip-off with crappy food."],
    "4": ["When you spoke at the meeting", " everyone was hanging onto your every word.", " I swear it was the most boring crap I've ever heard."],
    "5": ["The new pet you got", " is absolutely adorable and friendly.", " is a nasty little devil."],
    "6": ["The poem you wrote", " captures the essence of love beautifully.", " reads like a fifth-grader's damn joke."],
    "7": ["Your recommendation for the hiking trail", " led to breathtaking views.", " was the worst and most miserable trek ever."],
    "8": ["The restaurant you rave about", " truly offers a transcendent culinary experience.", " gave me food poisoning, for God's sake."],
    "9": ["Your gardening skills", " have transformed our backyard into a paradise.", " turned it into a hellish wasteland."],
    "10": ["The workout routine you follow", " is dynamic and seems really effective.", " looks like a torturous hell regimen."]
}
prompts=[]
love_completions=[]
hate_completions=[]
for key, example in Loving_Hating_dict.items():
    prompts.append(example[0])
    love_completions.append(example[1])
    hate_completions.append(example[2])

In [20]:
resolution=10
#when you have your steerig vector(here for example: "British" - "American"), do a grid search over layers and coefficents
#this might take a while
weighted_steering_prompts={" love":1," hate":-1}
layer_list=[int(i) for i in np.linspace(1,len(model.model.layers)-1,resolution)]
coefficent_list= [i for i in np.linspace(0,10,resolution,dtype=int)]

love_hate_results=adhoc_hyperparameter_search.layer_coefficient_gridsearch(
    model,
    tokenizer,
    prompts,
    weighted_steering_prompts,
    layer_list,
    coefficent_list,
    love_completions,
    hate_completions
)

Processing: 100%|██████████| 100/100 [06:59<00:00,  4.20s/it]


In [21]:
wanted_matrix, unwanted_matrix=adhoc_hyperparameter_search.create_perplexity_matrices(love_hate_results)
adhoc_hyperparameter_search.plot_matrices_with_pareto(love_hate_results)
adhoc_hyperparameter_search.plot_pareto_frontier(love_hate_results)

In [ ]:
resolution=10
#when you have your steerig vector(here for example: "British" - "American"), do a grid search over layers and coefficents
#this might take a while
weighted_steering_prompts={" love":1," hate":-1}
layer_list=[int(i) for i in np.linspace(1,10,resolution)]
coefficent_list= [i for i in np.linspace(0,10,resolution,dtype=int)]

love_hate_zoomed_results=adhoc_hyperparameter_search.layer_coefficient_gridsearch(
    model,
    tokenizer,
    prompts,
    weighted_steering_prompts,
    layer_list,
    coefficent_list,
    love_completions,
    hate_completions
)

In [23]:
[int(i) for i in np.linspace(1,10,resolution)]

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

In [14]:
from activation_additions import adhoc_actadds

num_layers = 8
residual_differences = {}

for injection_layer in range(num_layers):
    steered_residuals = {}
    unsteered_residuals = {}
    
    # Hook to modify the residual stream
    def test_hook(_, inpt):
        (resid_pre,) = inpt
        if resid_pre.shape[1] == 1:
            return
        resid_pre = resid_pre.to(model.device)
        resid_pre += t.randn_like(resid_pre)
    
    # Hook to save the residual stream
    def save_residual_hook(_, inpt):
        (resid_pre,) = inpt
        current_residuals[observation_layer] = resid_pre.detach().clone()
    
    tokens = tokenizer("I really feel like having a", return_tensors="pt")["input_ids"].to(model.device)
    injection_layer_name = adhoc_actadds.get_blocks(model)[injection_layer]
    
    # Steered pass
    current_residuals = {}
    with adhoc_actadds.pre_hooks(hooks=[(injection_layer_name, test_hook)]):
        for observation_layer in range(num_layers):
            observation_layer_name = adhoc_actadds.get_blocks(model)[observation_layer]
            with adhoc_actadds.pre_hooks(hooks=[(observation_layer_name, save_residual_hook)]):
                _ = accelerator.unwrap_model(model.forward(tokens))
    steered_residuals = current_residuals.copy()
    
    # Unsteered pass
    current_residuals = {}
    for observation_layer in range(num_layers):
        observation_layer_name = adhoc_actadds.get_blocks(model)[observation_layer]
        with adhoc_actadds.pre_hooks(hooks=[(observation_layer_name, save_residual_hook)]): 
            _ = accelerator.unwrap_model(model.forward(tokens))
    unsteered_residuals = current_residuals.copy()

    # Calculate differences for all observation layers given an injection layer
    differences = {}
    for observation_layer in range(num_layers):
        difference = (steered_residuals[observation_layer] - unsteered_residuals[observation_layer]).abs().mean().item()
        differences[observation_layer] = difference
    residual_differences[injection_layer] = differences

# Display results
for injection_layer, differences in residual_differences.items():
    print(f"\nInjection Layer: {injection_layer}")
    for observation_layer, difference in differences.items():
        print(f"Observation Layer {observation_layer}: Mean Abs Difference = {difference}")



Injection Layer: 0
Observation Layer 0: Mean Abs Difference = 0.7970435619354248
Observation Layer 1: Mean Abs Difference = 0.8020190000534058
Observation Layer 2: Mean Abs Difference = 0.8017208576202393
Observation Layer 3: Mean Abs Difference = 0.8764550685882568
Observation Layer 4: Mean Abs Difference = 0.8798004388809204
Observation Layer 5: Mean Abs Difference = 0.8951536417007446
Observation Layer 6: Mean Abs Difference = 0.9356597065925598
Observation Layer 7: Mean Abs Difference = 0.9521691799163818

Injection Layer: 1
Observation Layer 0: Mean Abs Difference = 0.0
Observation Layer 1: Mean Abs Difference = 0.8000084161758423
Observation Layer 2: Mean Abs Difference = 0.793964147567749
Observation Layer 3: Mean Abs Difference = 0.8740732073783875
Observation Layer 4: Mean Abs Difference = 0.8717744946479797
Observation Layer 5: Mean Abs Difference = 0.8990942239761353
Observation Layer 6: Mean Abs Difference = 0.931395411491394
Observation Layer 7: Mean Abs Difference = 0.94

In [38]:
from activation_additions import analysis_automatic
from importlib import reload

# Reload the modules
analysis_automatic = reload(analysis_automatic)


In [39]:
import pandas as pd

# Sample DataFrame
data = {
    "prompts": [
        "My neighbor always", 
        "Dogs are", 
        "The concert last night",
        "Your cooking"
    ],
    "completions": [
        "gives me the best advice and helps in tough times.",
        "just disgusting creatures that always bark.",
        "was an absolute delight, every performance touched my heart.",
        "tastes like someone dumped garbage in a pot."
    ],
    "is_modified": [False, True, False, True]
}

df = pd.DataFrame(data)



In [40]:
analysis_results=analysis_automatic.rate_completions_openai(
    data_frame=df, criterion="expresses positive emorions"
)

Cost upper bound: $0.008
0 10
1 0
2 10
3 0
Average ratings:
is_modified
False    10.0
True      0.0
Name: rating, dtype: float64


In [47]:
df["rating_text"][2]

'\n\nThis text expresses very positive emotions. It conveys a feeling of joy and appreciation for the performances. The rating is 10.'

In [43]:
df

,prompts,completions,is_modified,rating,rating_text
0,My neighbor always,gives me the best advice and helps in tough ti...,False,10.0,\n\nThis text expresses a very positive emotio...
1,Dogs are,just disgusting creatures that always bark.,True,0.0,\n\nThis text does not express any positive em...
2,The concert last night,"was an absolute delight, every performance tou...",False,10.0,\n\nThis text expresses very positive emotions...
3,Your cooking,tastes like someone dumped garbage in a pot.,True,0.0,\n\nThis text does not express any positive em...
